In [1]:
import numpy as np 
from scipy.stats import norm
from scipy.linalg import cholesky

$$
\mathbb{P}\left(Y \in \cdot \mid Y \in \mathrm{S}_{\mu^{(t)}, \mathbf{i}}\right) \text { et } \mathbb{P}\left(Y \in \cdot \mid\left[\mu^{(t)}\right]^T Y=s\right)
$$
On le réécrit : 
$$
X=v^{\top} \xi \sim N\left(0, v^{\top} \Sigma v\right)=N(0,1)
$$
On a la distrbution conditionnelle : 

$$
(\xi \mid X=x) \sim N\left(\frac{\Sigma v}{v^{\top} \Sigma v} x, \Sigma-\frac{\Sigma v v^{\top} \Sigma}{v^{\top} \Sigma v}\right)=N\left(\Sigma v x, \Sigma-\Sigma v v^{\top} \Sigma\right)
$$
On utilise le tips ci-dessous pour pouvoir simuler (décomposition du terme en variance)

$$
\begin{aligned}
& \left(A-\Sigma v v^{\top} A\right)\left(A-\Sigma v v^{\top} A\right)^{\top} \\
& \quad=A A^{\top}-A A^{\top} v v^{\top} \Sigma-\Sigma v v^{\top} A A^{\top}+\Sigma v v^{\top} \Sigma v v^{\top} \Sigma \\
& \quad=\Sigma-\Sigma v v^{\top} \Sigma
\end{aligned}
$$

Suivant que l'on veuille sample avec ou non une condition sur la valeur de X, on le spécifie dans la fonction ci-dessous.

$$
\begin{aligned}
& \text { for } i=1, \ldots, K \\
& \quad \text { generate } U \sim \operatorname{Unif}[0,1] \\
& \quad V \leftarrow(i-1+U) / K \\
& \quad X \leftarrow \Phi^{-1}(V) \\
& \quad \text { generate } Z \sim N(0, I) \text { in } \Re^d \\
& \quad \xi \leftarrow \Sigma v X+\left(A-\Sigma v v^{\top} A\right) Z
\end{aligned}
$$

In [2]:
def stratified_sampling_linear_projection(mu, Sigma, v, K,x=None):
    """
    Generates K samples from N(0, Sigma) stratified along the direction determined by v.
    
    Args:
    - mu (array): The mean vector of the distribution.
    - Sigma (2D array): The covariance matrix of the distribution.
    - v (array): The vector along which stratification is done.
    - K (int): The number of stratified samples to generate.
    - x (optionnal) :  si on donne une valeur pour x alors c'est qu'on conditionne à une valeur particulière de x. Sinon on sample sur tout la strat Si. Taille K
    
    Returns:
    - samples (2D array): The generated stratified samples. (K, de N(0,sigma), stratifié sur la direction donnée par v)
    """
    d = len(mu)  # Dimension of the normal distribution
    # Normalize v so that v^T Sigma v = 1
    v = v / np.sqrt(v.T @ Sigma @ v)
    
    # Generate K stratified samples for the standard normal distribution along v
    U = np.random.uniform(0, 1, K)     # Uniformly distributed samples in (0,1)
    V = (np.arange(K) + U) / K         # Stratified samples in (0,1)
    X = norm.ppf(V)                    # Inverse CDF (quantile function) to get stratified samples for N(0, 1)
   
    # Compute the matrix A for the conditional distribution of xi given x
    A = cholesky(Sigma, lower=True)    # Cholesky factorization
    A_minus_v_Sigma_v_T_A = A - np.outer(Sigma @ v, v.T @ A)
    
    # Generate K samples from the conditional distribution of xi given X
    Z = np.random.randn(K, d)  # Z ~ N(0, I) in d dimensions 
    if x is not None:

        xi_samples = Sigma @ v * x[:, None] + (A_minus_v_Sigma_v_T_A @ Z.T).T  # Conditional samples 

    else:
        xi_samples = Sigma @ v * X[:, None] + (A_minus_v_Sigma_v_T_A @ Z.T).T 
        
    
    return xi_samples + mu             # Add the mean to each sample


In [3]:
# Example usage:
mu = np.array([1, 2, 3])               # Mean vector - on est en 3 dimensions 
Sigma = np.array([[1, 0.5, 0.1],       # Covariance matrix
                  [0.5, 2, 0.3],
                  [0.1, 0.3, 1]])
v = np.array([1, 1, 1])                # Vector for stratification
K = 10                            # Number of samples to generate - 10 strates. Donc pour chaque strat les 3 coordonnées. 

# Generate stratified samples
samples = stratified_sampling_linear_projection(mu, Sigma, v, K)
#samples  # Display first 5 samples for brevity


In [4]:
samples

array([[-1.11281429,  0.20791618,  1.50514109],
       [ 1.46661494,  0.05736931,  2.07885552],
       [ 0.71354495,  1.20057925,  2.30792473],
       [ 1.85319342,  0.72941979,  2.43164639],
       [ 1.06523413,  0.83838845,  3.51691007],
       [ 0.93717706,  3.34242541,  2.13840583],
       [ 0.89990674,  3.182639  ,  2.79840738],
       [ 2.13809686,  2.10785281,  3.71004494],
       [ 2.66272563,  3.77733989,  1.99310531],
       [ 1.62423839,  2.96071703,  4.66203523]])